Links I used

1. Inspiration
http://tlfvincent.github.io/2015/10/23/presidential-speech-topics/
    
2. Understand topic modelling
https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

3. Helpsheet for LDA - quite interesting, he applied topic modelling for the whole wiki page
https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation
    
4. Cheatsheet
https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

Open up the file containing LHL's speech, txt file

In [2]:
# open the files
filename = "lhlspeech.txt"
txt = open(filename,"r")
txt = txt.read()

Using nltk module, import sent_tokenize function
<br>Create tokens by sentences
<br>Check length of this token of sentences
<br>LHL speech contains 622 sentences


In [3]:
#tokenize txt in terms of sentences
from nltk.tokenize import sent_tokenize
sent_tokenize_list = sent_tokenize(txt)
len(sent_tokenize_list)

622

Download these packages to plot

In [4]:
# for visualisation
#!pip install gensim
from gensim import matutils
import matplotlib.pyplot as plt

c:\users\annabel\anaconda2\envs\pydot_p3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
import nltk
import re

Create a list from the tokenized sentences (list)
<br> Extract out individual sentences by creating a dictionary

In [6]:
#creating a list  from the text
doc = {}
for i,sent in enumerate(sent_tokenize_list):
    doc["doc"+str(i)] = sent.replace('\n',' ').strip()
    #print ("")

Test and see if each sentence is captured individually

In [7]:
doc["doc10"]

'In Asia, I expect many countries will continue to rise and especially the biggest ones - China and India.'

Return the dictionary values, which captures each sentence individually, as a list

In [8]:
sentenses = doc.values() #take in the values from the dictionary

You can tell from the way each sentence is a separate string, that the sentences are separated

In [9]:
print (sentenses)

dict_values(['We have travelled from Third World to First.', 'You know the Singapore story to date well.', 'Our question is what is the next chapter of this story?', 'Where do we want Singapore to be 20 years from now?', 'The next 20 years will see many changes in the world.', 'The first big question will be: will there be peace or instability in the world, and in Asia?', 'If it is not peaceful, if there are tensions, then we must brace ourselves for a rough ride.', 'But if there is peace and prosperity worldwide, which looks likely, then it is going to be an exciting era of rapid progress and dramatic change.', 'We are both a country and a city.', 'So when we look forward into our future, we have to see our future against what other countries are doing and also against what other cities are doing in the world.', 'In Asia, I expect many countries will continue to rise and especially the biggest ones - China and India.', 'And they will compete against us in many business areas - manufac

In [10]:
type(sentenses)

dict_values

I copied template from this website
<br>https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
Clean tokens in these steps:
1. create tokens
2. create stop words
3. remove punctuation
4. return words to root using lemmatizer

Specific to this case I removed words like "Singapore" because probably very common in a national day speech
<br>I also removed words less than 4 letters (not very useful)
<br>You might have to keep editing the stopwords based on the results you see later

In [38]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = stopwords.words('english') + ["singapore","singaporeans","singaporean","country","singapura","countries"]
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    #take in lower case tokens and only take in tokens that are at least 5 tokens
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #cut_words = "".join([i for i in stop_free if len(i)>5])
    #exclude punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #lemmtatize the words
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in sentenses]   

In [39]:
print (doc_clean)

[['travelled', 'third', 'world', 'first'], ['know', 'story', 'date', 'well'], ['question', 'next', 'chapter', 'story'], ['want', '20', 'year', 'now'], ['next', '20', 'year', 'see', 'many', 'change', 'world'], ['first', 'big', 'question', 'be', 'peace', 'instability', 'world', 'asia'], ['peaceful', 'tension', 'must', 'brace', 'rough', 'ride'], ['peace', 'prosperity', 'worldwide', 'look', 'likely', 'going', 'exciting', 'era', 'rapid', 'progress', 'dramatic', 'change'], ['city'], ['look', 'forward', 'future', 'see', 'future', 'also', 'city', 'world'], ['asia', 'expect', 'many', 'continue', 'rise', 'especially', 'biggest', 'one', 'china', 'india'], ['compete', 'u', 'many', 'business', 'area', 'manufacturing', 'service'], ['rd', 'area', 'surpass', 'u', 'part', 'china', 'india', 'certainly', 'surpass', 'u', 'time', 'big', 'giant', 'offer', 'many', 'opportunity', 'singaporean'], ['city', 'shanghai', 'beijing', 'mumbai', 'bangalore', 'become', 'even', 'vibrant', 'exciting', 'place', 'live', 'w

These are the packages needed for topic modelling
<br>https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

First you create a dictionary of all the values, every unique term is assigned an index
<br>The values here do not need to be unique, in fact you don't want it to be unique, else canot count the frequency
<br>corpora.Dictionary knows to take it only unique values
<br>Then, count the number of times this word appears in the cleaned data
<br>Return a tupule of the (word_id,times it appears)

In [40]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
# which converts a collection of words to its bag-of-words representation
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Initialise the lda model, to Lda (easier to call this function later)
<br>Note that the lda model is from gensim, hence imported gensim earlier
<br> Initialise the number of topics you want the model to "capture" to be 5
<br> Passes = number of times you pass over the dictionary

In [41]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
speeches_topics = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

Print the topics inferred from the speech and the most important x words
<br>Initiatlise number of topics to be 5

In [42]:
print(speeches_topics.print_topics(num_topics=5, num_words=3))

[(0, '0.013*"year" + 0.008*"one" + 0.008*"time"'), (1, '0.015*"one" + 0.012*"think" + 0.011*"many"'), (2, '0.010*"part" + 0.009*"family" + 0.007*"one"'), (3, '0.017*"people" + 0.009*"take" + 0.009*"think"'), (4, '0.021*"one" + 0.008*"people" + 0.007*"time"')]


Install and pyldavis for interactive visualisation

In [ ]:
#!pip install pyldavis

In [43]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

Create the interactive visualisation
See the 5 main topics and the proportion of words assigned to this topic

In [44]:
vis_data = gensimvis.prepare(speeches_topics, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

c:\users\annabel\anaconda2\envs\pydot_p3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
